In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
pd.options.display.max_columns = 195

In [3]:
df=pd.read_csv("data/Speed+Dating+Data.csv", encoding='ISO-8859-1')

## What's in the Dataset

- A very large dataset of 8378 rows and 195 columns
- Each row represents a speed date between a man and a woman during an event.
- There were 21 events, called "waves," held between October 2002 and April 2004.
- During each wave, participants met between 5 and 22 different people ("rounds").
- A total of 551 unique individuals participated in at least one event. 
- At each event, the male-to-female ratio was almost always balanced: 277 men and 274 women.
- Men attendeed a total of 274 seeed dates.
- Women attendeed a total of 277 speed dates.
- 16.4% of the speed dates resulted in a match.

At 4 times, the participants are asked to answer a survey in order to give informations about themself and about the event: 
1. **Before attending the event (Signup/Time1) :** demographics, dating habits, self-perception across key attributes (Attractive, Sincere, Intelligent, Fun, Ambitious, Shared Interest), beliefs on what others find valuable in a mate, how important these attribues are for them in a date, lifestyle information...
2. **During the event (Scorecard) :** after each speed date, the participants secretly answer if they would go on a second date with their partner and evaluate them on key attributes. Half way through meeting all potential dates during the night, the participants are asked again about the importance of key attribues in a date and how they percieve themself.
3. **A day after the event (Followup/Time2) :** general satisfaction about the people they met, global question about the speed dating event. Once again, evaluation of key attributes (what's important for them, for other, self-perception)
4. **3-4 weeks after the event (Followup/Time3) :** what happend after the event (second date?). One last time, evaluation of key attributes (what's important for them, for other, self-perception)



In [49]:
df.shape

(8378, 196)

In [50]:
df.describe(include= "all")

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,...,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,match_label
count,8378.000000,8377.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,6532.000000,8378.000000,8378.000000,8368.000000,8378.000000,8220.000000,8378.000000,8274.000000,8305.000000,8289.000000,8289.000000,8289.000000,8280.000000,8271.000000,8249.000000,8378.000000,8166.000000,8091.000000,8072.000000,8018.000000,7656.000000,7302.000000,8128.000000,8060.000000,7993.000000,8283.000000,8315,8296.000000,4914,3133,3583,8315.000000,8299.000000,8299.000000,8299,7314,4279,8299.000000,8281.000000,8299.000000,8289,8240.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8299.000000,8277.000000,1800.000000,8299.000000,8299.000000,8299.000000,8289.000000,8279.000000,8257.000000,6489.000000,6489.000000,6489.000000,6489.000000,6489.000000,6467.000000,8299.000000,8299.000000,8299.000000,8299.000000,8289.000000,8289.000000,8273.000000,8273.000000,8273.000000,8273.000000,8273.000000,4906.000000,4906.000000,4906.000000,4906.000000,4906.000000,...,8101.000000,8082.000000,8028.000000,7666.000000,7311.000000,8138.000000,8069.000000,8003.000000,7205.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4096.000000,4000.00000,4000.000000,4000.000000,4000.000000,4000.000000,7463.000000,7463.000000,7433.000000,1984.000000,1955.000000,1984.000000,1984.000000,1955.000000,1974.000000,7445.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.000000,5775.00000,5775.000000,5775.000000,5775.000000,5775.000000,7463.000000,7463.000000,7463.000000,7463.000000,7463.000000,4377.000000,4377.000000,4377.000000,4377.000000,4377.000000,3974.000000,3974.000000,3974.000000,1496.000000,668.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2959.000000,2016.000000,3974.000000,3974.000000,3974.000000,3974.000000,3974.000000,2016.000000,2016.000000,2016.000000,2016.000000,2016.000000,8378
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259,NaN,241,68,115,NaN,NaN,NaN,269,409,261,NaN,NaN,NaN,367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [51]:
rounds_per_wave = df.groupby("wave")["round"].mean().reset_index()

fig = px.bar(
    rounds_per_wave,
    x="wave",
    y="round",
    labels={"wave": "Waves", "round": "Number of rounds"},
    title="Number of rounds per wave",
    hover_data={"wave": True, "round": True}
)

fig.show()

In [53]:
df_unique= df.drop_duplicates(subset='iid').copy()
print(f'Number of unique participants : {df_unique.shape[0]}')

Number of unique participants : 551


In [58]:
# A balanced male-to-female ratio
df_unique["gender"].value_counts()

gender
1    277
0    274
Name: count, dtype: int64

In [116]:
# A balanced male-to-female ratio at almost each wave

gender_counts = df_unique.groupby(["wave", "gender"]).size().unstack(fill_value=0).reset_index()
gender_counts.columns = ['wave', 'Women', 'Men']

fig = px.bar(
    gender_counts,
    x="wave",
    y=["Women", "Men"],
    labels={"wave": "Wave", "value": "Number of participants", "variable": "Gender"},
    title="Number of participants per wave and per gender <br><sub>Base: 277 men, 274 women.",
    barmode='group',
    color_discrete_map={'Women': 'red', 'Men': 'blue'}
)

fig.update_layout(
    xaxis_title='Waves',
    yaxis_title='Number of participants',
    legend_title='Gender'
)

fig.show()


In [62]:
pivot_df = df_unique.pivot_table(index='wave', columns='gender', values='round', aggfunc='count', fill_value=0)

pivot_df.columns = ['Number of round for Men', 'Number of round for Women']

print("Total number of round for men",pivot_df['Number of round for Men'].sum())
print("Total number of round for women",pivot_df['Number of round for Women'].sum())


Total number of round for men 274
Total number of round for women 277


In [118]:
# Map match values to alphabetic values
match_mapping = {1: 'Match', 0: 'No Match'}
df['match_label'] = df['match'].map(match_mapping)

# Calculate the counts of each match type
match_counts = df["match_label"].value_counts().reset_index()
match_counts.columns = ['Match Type', 'Count']

# Calculate the total count
total_count = match_counts['Count'].sum()

# Calculate the percentages
match_counts['Percentage'] = (match_counts['Count'] / total_count) * 100

# Create a pie chart
fig = px.pie(
    match_counts,
    values='Count',
    names='Match Type',
    title='Match Distribution <br><sub>Base: 551 participants. 277 speed dates.',
    hover_data={'Percentage': True},
    labels={'Percentage': 'Percentage'},
)

# Update layout for better readability and adjust size
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    marker=dict(colors=['lightgrey', 'green'])
)
fig.update_layout(
    width=600,  
    height=400  
)

# Show the plot
fig.show()


In [68]:
match_per_wave = df.groupby("wave")[["match", "round"]].mean().reset_index()
match_per_wave["match"] = match_per_wave["match"].round(2)

match_per_wave["round_color"] = np.where(
    match_per_wave["round"] <= 10,
    "Limited choice (≤10 round)",
    "Extensive choice (>10 round)"
)

fig = px.bar(
    match_per_wave,
    x="wave",
    y="match",
    color="round_color",  
    color_discrete_map={
        "Limited choice (≤10 round)": "lightgreen",
        "Extensive choice (>10 round)": "darkgreen"
    },
    labels={
        "wave": "Waves",
        "match": "Percentage of match",
        "round_color": "Number of rounds"
    },
    title="Percentage of match per wave <br><sub>Base: 551 participants. 277 speed dates.",
    hover_data={"wave": True, "match": True, "round": True}
)

fig.show()

The amount of rounds -people met during the night- doens't influence the possibility of a match.

## Missing values analysis

In [69]:
columns_with_na = df.columns[(df.isna().sum() / len(df)).round(3) != 0]
print(f"{df[columns_with_na].shape[1]} columns with missing values, {((df[columns_with_na].shape[1])/len(df.columns))*100:.2f}% of the total number of columns")


181 columns with missing values, 92.35% of the total number of columns


In [70]:
# Filter the dataset on columns without NAN
percentage_na = (df.isna().sum() / len(df)).round(3)
columns_without_na = percentage_na[percentage_na == 0]

result_df_without_NA = pd.DataFrame({
    'Column Name': columns_without_na.index,
    'NA Rate': columns_without_na.values
})

print("columns without NAN:")
print(result_df_without_NA)

columns without NAN:
    Column Name  NA Rate
0           iid      0.0
1            id      0.0
2        gender      0.0
3           idg      0.0
4        condtn      0.0
5          wave      0.0
6         round      0.0
7      position      0.0
8         order      0.0
9       partner      0.0
10        match      0.0
11     samerace      0.0
12        dec_o      0.0
13          dec      0.0
14  match_label      0.0


The columns without missing values almost exclusively correspond to information **essential** for the speed dating event:

- Participant identifiers (IDs, gender, partner ID, whether the partners are from the same race)
- Information about the event (wave, number of rounds, participant's position, order of appearance, whether participants will meet more or fewer than 10 potential partners during the evening)
- Information about the outcome of the date (match, participant’s decision, partner’s decision)

In [71]:
# Filter the dataset on columns with NAN
percentage_na = (df.isna().sum() / len(df)).round(3)
columns_with_na = percentage_na[percentage_na != 0]

result_df_with_NA = pd.DataFrame({
    'Column Name': columns_with_na.index,
    'NA Rate': columns_with_na.values
})

# Order the DataFrame by 'NA Rate'
result_df_with_NA_sorted = result_df_with_NA.sort_values(by='NA Rate', ascending=False)

print("Columns with NAN (ordered by NAN rate) :")
print(result_df_with_NA_sorted)

Columns with NAN (ordered by NAN rate) :
    Column Name  NA Rate
146    num_in_3    0.920
145    numdat_3    0.821
55       expnum    0.785
112      amb7_2    0.767
109     sinc7_2    0.767
..          ...      ...
38     tvsports    0.009
37       sports    0.009
21        field    0.008
26         race    0.008
1           pid    0.001

[181 rows x 2 columns]


Although many columns contain missing values, those least frequently completed correspond to:

- **the potential follow-up of the dates** (whether additional meetings occurred after the event, and who initiated contact),
- **the participant’s self-assessed desirability** (how many people expressed interest in their profile),
- **the follow-up evaluations** conducted one day (Time2) or three to four weeks (Time3) after the event, regarding the partners they met, themselves, or characteristics perceived as important by the opposite gender,
- **academic performance, education level, and income.**

# Participant profil analysis

- Participants attending are, on average, 26 to 27 years old, with a similar distribution between men and women.
- There is a similar distribution in terms of race/ethnic background, with slightly more European/Caucasian-Americans among men.
- A large number of men work in business/economy/finance (followed by engineering, law, or biological sciences/chemistry/physics).
- Women, on the other hand, more often work in social sciences/psychology, education, or social work—fields where men are less present.

- Men and women attending the event go out with the same frequency—at least twice a week. However, women tend to go on dates slightly less often.
- Most participants of both sexes go on a date at least once a month.


In [81]:
print("average age Men: ",df_unique[df_unique["gender"]==1]["age"].mean().round(0))
print("average age women: ",df_unique[df_unique["gender"]==0]["age"].mean().round(0))
print("average age total:",df_unique["age"].mean().round(0))

average age Men:  27.0
average age women:  26.0
average age total: 26.0


In [119]:
# Gender mapping
gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Initialize the figure
fig = go.Figure()

# Add traces for Women
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Women']['age'],
        name='Women',
        marker_color='red',
    ),
)

# Add traces for Men
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Men']['age'],
        name='Men',
        marker_color='blue',
    ),
)

# Update layout with correct Y axis scale
fig.update_layout(
    title_text="Age of participants <br><sub>Base: 277 men, 274 women.",
    height=600,
    width=800,
)

fig.show()


In [120]:
# Mapping for race and gender
race_mapping = {
    1: 'Black/African American',
    2: 'European/Caucasian-American',
    3: "Latino/Hispanic American",
    4: "Asian/Pacific Islander/Asian-American",
    5: 'Native American',
    6: 'Other'
}
df_unique["Participant's_race"] = df_unique['race'].map(race_mapping)

gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Grouping by race and gender
race_gender_counts = df_unique.groupby(['Participant\'s_race', 'gender_label']).size().reset_index(name='count')

# Calculating percentages
total_men = len(df_unique[df_unique['gender_label'] == 'Men'])
total_women = len(df_unique[df_unique['gender_label'] == 'Women'])

race_gender_counts['percentage'] = race_gender_counts.apply(
    lambda row: f"{row['count'] / total_men * 100:.0f}%" if row['gender_label'] == 'Men'
    else f"{row['count'] / total_women * 100:.0f}%",
    axis=1
)

# Sort the data by total number of participants (men + women)
race_gender_counts_total = race_gender_counts.groupby('Participant\'s_race')['count'].sum().reset_index()
race_gender_counts_total = race_gender_counts_total.sort_values('count', ascending=False)

# Reorder data for men and women by the sorted race order
race_gender_counts = race_gender_counts.set_index('Participant\'s_race').loc[race_gender_counts_total['Participant\'s_race']].reset_index()

# Create the plot
fig = go.Figure(data=[
    go.Bar(name='Women', x=race_gender_counts[race_gender_counts['gender_label'] == 'Women']['Participant\'s_race'],
           y=race_gender_counts[race_gender_counts['gender_label'] == 'Women']['count'],
           text=race_gender_counts[race_gender_counts['gender_label'] == 'Women']['percentage'],
           textposition='outside', marker_color='red'),
    go.Bar(name='Men', x=race_gender_counts[race_gender_counts['gender_label'] == 'Men']['Participant\'s_race'],
           y=race_gender_counts[race_gender_counts['gender_label'] == 'Men']['count'],
           text=race_gender_counts[race_gender_counts['gender_label'] == 'Men']['percentage'],
           textposition='outside', marker_color='blue')
])

# Update layout
fig.update_layout(
    barmode='group',
    title='Distribution of race per gender <br><sub>Base: 277 men, 274 women.',
    xaxis_title='Race',
    yaxis_title='Number of participants',
    width=900,
    height=600
)

# Show the plot
fig.show()


In [86]:
field_mapping = {
    1: 'Law',
    2: 'Math',
    3: "Social Science, Psychologist",
    4: "Medical Science, Pharmaceuticals, and Bio Tech",
    5: 'Engineering',
    6: 'English/Creative Writing/ Journalism',
    7: 'History/Religion/Philosophy',
    8: 'Business/Econ/Finance',
    9: 'Education, Academia',
    10: 'Biological Sciences/Chemistry/Physics',
    11: 'Social Work',
    12: 'Undergrad/undecided',
    13: 'Political Science/International Affairs',
    14: 'Film',
    15: 'Fine Arts/Arts Administration',
    16: 'Languages',
    17: 'Architecture',
    18: 'Other'
}

df_unique["Participant's_field"] = df_unique['field_cd'].map(field_mapping)

field_counts = df_unique["Participant's_field"].value_counts().reset_index()
field_counts.columns = ["Participant's_field", "count"]

field_counts = field_counts.sort_values('count', ascending=False)

fig = go.Figure(data=[
    go.Bar(
        x=field_counts["Participant's_field"],
        y=field_counts["count"],
        text=field_counts["count"],
        textposition='outside',
        marker_color='mediumpurple'
    )
])

fig.update_layout(
    title="Participants' field distribution <br><sub>Base: 551 participants.",
    xaxis_title="Field",
    yaxis_title="Number of participants",
    width=900,
    height=600
)

fig.show()


In [124]:
gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Compute counts for each combination of field and gender
field_gender_counts = df_unique.groupby(["Participant's_field", 'gender_label']).size().reset_index(name='count')

# --- Sort fields by total participants (Men + Women) ---
field_total_counts = field_gender_counts.groupby("Participant's_field")['count'].sum().reset_index()
sorted_fields = field_total_counts.sort_values('count', ascending=False)["Participant's_field"].tolist()

# Create the figure
fig = go.Figure(data=[
    go.Bar(name='Women', 
           x=field_gender_counts[field_gender_counts['gender_label'] == 'Women']["Participant's_field"],
           y=field_gender_counts[field_gender_counts['gender_label'] == 'Women']['count'],
           text=field_gender_counts[field_gender_counts['gender_label'] == 'Women']['count'],  
           textposition='outside', marker_color='red'),
    
    go.Bar(name='Men', 
           x=field_gender_counts[field_gender_counts['gender_label'] == 'Men']["Participant's_field"],
           y=field_gender_counts[field_gender_counts['gender_label'] == 'Men']['count'],
           text=field_gender_counts[field_gender_counts['gender_label'] == 'Men']['count'],  
           textposition='outside', marker_color='blue')
])

# Update layout to force the correct X-axis order
fig.update_layout(
    barmode='group',
    title="Participants' field distribution (ordered by total number of participants per field) <br><sub>Base: 277 men, 274 women.",
    yaxis_title='Number of participants',
    xaxis=dict(
        categoryorder='array',
        categoryarray=sorted_fields
    ),
    width=900, 
    height=600  
)

fig.show()



In [82]:
print("Mean frequency of date :") 
print('Men :', df_unique[df_unique['gender']==1]["date"].mean())
print('Women :', df_unique[df_unique['gender']==0]["date"].mean())
print()
print('Median frequency of date :') 
print('Men :', df_unique[df_unique['gender']==1]["date"].median())
print('Women :', df_unique[df_unique['gender']==0]["date"].median())

Mean frequency of date :
Men : 4.835766423357664
Women : 5.133828996282528

Median frequency of date :
Men : 5.0
Women : 5.0


In [123]:
date_mapping = {
    1: "Several times a week",
    2: "Twice a week",
	3: "Once a week",
	4: "Twice a month",
	5: "Once a month",
	6: "Several times a year",
	7: "Almost never"

}
df_unique["date_label"] = df_unique['date'].map(date_mapping)

gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

date_gender_counts = df_unique.groupby(['date_label', 'gender_label']).size().reset_index(name='count')

total_men = len(df_unique[df_unique['gender_label'] == 'Men'])
total_women = len(df_unique[df_unique['gender_label'] == 'Women'])

date_gender_counts['percentage'] = date_gender_counts.apply(
    lambda row: f"{row['count'] / total_men * 100:.0f}%" if row['gender_label'] == 'Men'
    else f"{row['count'] / total_women * 100:.0f}%",
    axis=1
)


# Initialize the figure
fig = go.Figure()
# Create the plot
fig = go.Figure(data=[
    go.Bar(
        name='Women', 
        x=date_gender_counts[date_gender_counts['gender_label'] == 'Women']['date_label'],
        y=date_gender_counts[date_gender_counts['gender_label'] == 'Women']['count'],
        text=date_gender_counts[date_gender_counts['gender_label'] == 'Women']['percentage'],
        textposition='outside', 
        marker_color='red'
    ),
    go.Bar(
        name='Men', 
        x=date_gender_counts[date_gender_counts['gender_label'] == 'Men']['date_label'],
        y=date_gender_counts[date_gender_counts['gender_label'] == 'Men']['count'],
        text=date_gender_counts[date_gender_counts['gender_label'] == 'Men']['percentage'],
        textposition='outside', 
        marker_color='blue'
    )
])

# Update layout to keep the order of the dates
fig.update_layout(
    barmode='group',
    title='Frequency of date per gender <br><sub>Base: 277 men, 274 women.',
    xaxis_title='Frequency of date',
    yaxis_title='Number of participants',
    xaxis=dict(
        categoryorder='array',
        categoryarray=["Several times a week", "Twice a week", "Once a week", 
                       "Twice a month", "Once a month", "Several times a year", "Almost never"]
    ),
    width=900, 
    height=600  
)


fig.show()

In [83]:
print("Mean frequency of going out :") 
print('Men :', df_unique[df_unique['gender']==1]["go_out"].mean())
print('Women :', df_unique[df_unique['gender']==0]["go_out"].mean())
print()
print('Median frequency of going out :') 
print('Men :', df_unique[df_unique['gender']==1]["go_out"].median())
print('Women :', df_unique[df_unique['gender']==0]["go_out"].median())


Mean frequency of going out :
Men : 2.1563636363636363
Women : 2.130111524163569

Median frequency of going out :
Men : 2.0
Women : 2.0


In [125]:
go_out_mapping = {
    1: "Several times a week",
    2: "Twice a week",
	3: "Once a week",
	4: "Twice a month",
	5: "Once a month",
	6: "Several times a year",
	7: "Almost never"

}
df_unique["go_out_label"] = df_unique['go_out'].map(go_out_mapping)

gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

go_out_gender_counts = df_unique.groupby(['go_out_label', 'gender_label']).size().reset_index(name='count')

total_men = len(df_unique[df_unique['gender_label'] == 'Men'])
total_women = len(df_unique[df_unique['gender_label'] == 'Women'])

go_out_gender_counts['percentage'] = go_out_gender_counts.apply(
    lambda row: f"{row['count'] / total_men * 100:.0f}%" if row['gender_label'] == 'Men'
    else f"{row['count'] / total_women * 100:.0f}%",
    axis=1
)


# Initialize the figure
fig = go.Figure()
# Create the plot
fig = go.Figure(data=[
    go.Bar(
        name='Women', 
        x=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Women']['go_out_label'],
        y=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Women']['count'],
        text=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Women']['percentage'],
        textposition='outside', 
        marker_color='red'
    ),
    go.Bar(
        name='Men', 
        x=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Men']['go_out_label'],
        y=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Men']['count'],
        text=go_out_gender_counts[go_out_gender_counts['gender_label'] == 'Men']['percentage'],
        textposition='outside', 
        marker_color='blue'
    )
])

# Update layout to keep the order
fig.update_layout(
    barmode='group',
    title='Frequency of going out per gender<br><sub>Base: 277 men, 274 women.',
    xaxis_title='Frequency of going out',
    yaxis_title='Number of participants',
    xaxis=dict(
        categoryorder='array',
        categoryarray=["Several times a week", "Twice a week", "Once a week", 
                       "Twice a month", "Once a month", "Several times a year", "Almost never"]
    ),
    width=900, 
    height=600  
)


fig.show()

## Attributes evaluation of themself

### Check of the answers accuracy :

Scales of attributes' evaluation have changed depending on the waves. In top of that, it seams that some questions have not been answered correctly by attendees (answers given out of range). Since it's not possible to analyse the answers of all participants together, some questions have been filtered.

In [94]:
df_unique_waves_69 = df_unique[(df_unique["wave"]>=6) & (df_unique["wave"]<=9)]
df_unique_waves_other = df_unique.drop(df_unique_waves_69.index)
df_unique_waves_1021 = df_unique[(df_unique["wave"]>=10) & (df_unique["wave"]<=21)]

In [97]:
# Waves 6-9: Please rate the importance of the following attributes in a potential date on a scale of 1-10 (1=not at all important, 10=extremely important):
# It looks like the question has not been answered correctly by the attendees in waves 6 to 9 since max values are out of range (1 to 10)
# For this reason, we will only consider waves 1 to 5 and 10 to 21 for this question
attributes = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
df_max_values = df_unique_waves_69[attributes + ['wave']].groupby('wave').max()
df_max_values

,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
wave,,,,,,
6,20.83,22.73,20.00,20.83,20.00,19.57
7,27.78,23.08,22.73,27.78,19.57,19.57
8,20.93,23.81,23.26,22.50,20.41,17.39
9,25.64,23.81,23.81,23.81,20.59,23.81


In [98]:
# Waves 1-5, 10-21: You have 100 points to distribute among the following attributes -- give more points to those attributes that are more important in a potential date, and fewer points to those attributes that are less important in a potential date.  Total points must equal 100.
# the attributes' rating seams globally correct
attributes = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
df_mean_values = df_unique_waves_other[attributes + ['wave']].groupby('wave').mean()
df_mean_values["total"] =  df_mean_values[attributes].sum(axis=1)
df_mean_values

,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,total
wave,,,,,,,
1,32.321000,11.114500,19.869000,19.614500,7.414500,9.665500,99.999000
2,24.088235,18.029412,23.352941,16.558824,9.352941,9.205882,100.588235
3,22.555556,15.666667,21.444444,18.722222,11.666667,12.111111,102.166667
4,19.750000,18.000000,22.083333,16.611111,10.416667,12.861111,99.722222
5,30.789474,15.526316,18.421053,19.166667,10.882353,8.235294,103.021156
10,22.444444,19.333333,18.666667,16.166667,10.333333,13.055556,100.000000
11,22.135000,19.254048,21.158810,16.754048,9.611190,11.087381,100.000476
12,21.523929,19.894643,21.066429,15.447143,10.117500,11.950714,100.000357
13,22.117647,17.411765,18.470588,17.117647,12.411765,12.470588,100.000000


### Analysis

Participants, both men and women, seem to have a generally good perception of themselves and believe that others have a generally good perception of them as well. There are no differences between men and women in this regard.

In [147]:
# Attributes own perception
attributes1 = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
men_means1 = df_unique[df_unique['gender'] == 1][attributes1].mean()
women_means1 = df_unique[df_unique['gender'] == 0][attributes1].mean()

# Attributes other perception's of them
attributes2 = ['attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1']
men_means2 = df_unique[df_unique['gender'] == 1][attributes2].mean()
women_means2 = df_unique[df_unique['gender'] == 0][attributes2].mean()

# Create figure with 1 row, 2 columns
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=['Perception of Self', 'Perception of Self by Others'],
    specs=[[{'type': 'polar'}, {'type': 'polar'}]]
)

# Add first plot (Perception of Self)
fig.add_trace(go.Scatterpolar(
    r=women_means1.values.tolist() + [women_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    mode='lines+markers',
    name='Women',
    marker_color='red'
), row=1, col=1)

fig.add_trace(go.Scatterpolar(
    r=men_means1.values.tolist() + [men_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    mode='lines+markers',
    name='Men',
    marker_color='blue'
), row=1, col=1)

# Add second plot (Perception of Self by Others)
fig.add_trace(go.Scatterpolar(
    r=women_means2.values.tolist() + [women_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    mode='lines+markers',
    name='Women',
    marker_color='red',
    showlegend=False  
), row=1, col=2)

fig.add_trace(go.Scatterpolar(
    r=men_means2.values.tolist() + [men_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    mode='lines+markers',
    name='Men',
    marker_color='blue',
    showlegend=False 
), row=1, col=2)

# Update layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10]  
        )
    ),
    polar2=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10]  
        )
    ),
    showlegend=True,  
    title=f"Comparison of Perceptions (Self vs. Others)<br><sup>Base all waves: 277 men, 274 women. 1=awful, 10=great</sup>",
    width=850,
    height=600
)

fig.show()



# Dating expectations

For most participants of both genders, attending the event is an oportunity to spend a fun night out or to meet new people. Only a few are hoping to get a date or find a serious relationship.

In [127]:
goal_mapping = {
    1: "Seemed like a fun night out",
	2: "To meet new people",
	3: "To get a date",
	4: "Looking for a serious relationship",
	5: "To say I did it",
	6: "Other"
}
df_unique["goal_label"] = df_unique['goal'].map(goal_mapping)

gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

goal_gender_counts = df_unique.groupby(['goal_label', 'gender_label']).size().reset_index(name='count')

total_men = len(df_unique[df_unique['gender_label'] == 'Men'])
total_women = len(df_unique[df_unique['gender_label'] == 'Women'])

goal_gender_counts['percentage'] = goal_gender_counts.apply(
    lambda row: f"{row['count'] / total_men * 100:.0f}%" if row['gender_label'] == 'Men'
    else f"{row['count'] / total_women * 100:.0f}%",
    axis=1
)

# Sort the goals based on the total count (men + women)
goal_gender_counts_total = goal_gender_counts.groupby('goal_label')['count'].sum().reset_index()
goal_gender_counts_total = goal_gender_counts_total.sort_values('count', ascending=False)
# Merge the total count data back into the original DataFrame to retain the sorting order
goal_gender_counts = goal_gender_counts.set_index('goal_label').loc[goal_gender_counts_total['goal_label']].reset_index()

# Initialize the figure
fig = go.Figure()
# Create the plot
fig = go.Figure(data=[
    go.Bar(
        name='Women', 
        x=goal_gender_counts[goal_gender_counts['gender_label'] == 'Women']['goal_label'],
        y=goal_gender_counts[goal_gender_counts['gender_label'] == 'Women']['count'],
        text=goal_gender_counts[goal_gender_counts['gender_label'] == 'Women']['percentage'],
        textposition='outside', 
        marker_color='red'
    ),
    go.Bar(
        name='Men', 
        x=goal_gender_counts[goal_gender_counts['gender_label'] == 'Men']['goal_label'],
        y=goal_gender_counts[goal_gender_counts['gender_label'] == 'Men']['count'],
        text=goal_gender_counts[goal_gender_counts['gender_label'] == 'Men']['percentage'],
        textposition='outside', 
        marker_color='blue'
    )
])

# Update layout
fig.update_layout(
    barmode='group',
    title='Distribution of goal per gender. <br><sub>Base: 277 men, 274 women.',
    xaxis_title='Goals',
    yaxis_title='Number of participants',
    width=900, 
    height=600  
)

fig.show()

Men have slightly higher expectations than women about the outcome of the envent

In [129]:
# Gender mapping
gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Initialize the figure
fig = go.Figure()

# Add traces for Women
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Women']['exphappy'],
        name='Women',
        marker_color='red',
    ),
)

# Add traces for Men
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Men']['exphappy'],
        name='Men',
        marker_color='blue',
    ),
)

# Update layout with correct Y axis scale
fig.update_layout(
    title_text="Expectation of being happy with the people met during the speed date event<br><sub>Base: 277 men, 274 women.",
    height=600,
    width=800,
    yaxis=dict(
        title='Expectation from 1 to 10',
        tickmode='linear',
        dtick=1,
        range=[0.5, 10.5]
))

fig.show()


...but women tend to be slightly more confident than men about how they could interest other participants during the event

In [130]:
# Gender mapping
gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Initialize the figure
fig = go.Figure()

# Add traces for Women
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Women']['expnum'],
        name='Women',
        marker_color='red',
    ),
)

# Add traces for Men
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Men']['expnum'],
        name='Men',
        marker_color='blue',
    ),
)

# Update layout with correct Y axis scale
fig.update_layout(
    title_text="Number of people the participant expect to be interested in dating them<br><sub>Base: 277 men, 274 women.",
    height=600,
    width=800,
    yaxis=dict(
        title='Expectation from 1 to 20',
        tickmode='linear',
        dtick=1,
        range=[-0.5, 20.5]
))

fig.show()


In [ ]:
# Add first plot 
fig.add_trace(go.Scatterpolar(
    r=women_means1.values.tolist() + [women_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    fill='toself',
    name='Women',
    marker_color='red'
), row=1, col=1)

fig.add_trace(go.Scatterpolar(
    r=men_means1.values.tolist() + [men_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    fill='toself',
    name='Men',
    marker_color='blue'
), row=1, col=1)

# Add second plot 
fig.add_trace(go.Scatterpolar(
    r=women_means3.values.tolist() + [women_means3.values[0]],  
    theta=attributes3 + [attributes3[0]],  
    fill='toself',
    name='Women',
    marker_color='red',
    showlegend=False  
), row=1, col=2)

fig.add_trace(go.Scatterpolar(
    r=men_means3.values.tolist() + [men_means3.values[0]],  
    theta=attributes3 + [attributes3[0]],  
    fill='toself',
    name='Men',
    marker_color='blue',
    showlegend=False 
), row=1, col=2)

# Add tird plot 
fig.add_trace(go.Scatterpolar(
    r=women_means2.values.tolist() + [women_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    fill='toself',
    name='Women',
    marker_color='red',
    showlegend=False  
), row=1, col=3)

fig.add_trace(go.Scatterpolar(
    r=men_means2.values.tolist() + [men_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    fill='toself',
    name='Men',
    marker_color='blue',
    showlegend=False 
), row=1, col=3)

# Dating priorisation


- For men, attractiveness is by far the most important attribute they are looking for.
- For women, the distribution is more balanced. Intelligence comes first, followed closely by attractiveness and fun.
- Participants' perceptions of what others of the same sex look for are quite different. Women believe that other women prioritize attractiveness first, with intelligence, ambition, and sincerity being less important.
- On the other hand, men believe that other men think like them, prioritizing attractiveness. However, they believe other men are also looking for an ambitious partner.
- Both genders agree that the opposite sex primarily looks for attractiveness, and their perceptions of the importance of other attributes are similar. The exception is ambition, where women believe this attribute is more important to men.


Finaly, dating in the same race or religious background is more important for women than for men.

In [149]:
# what you're looking for
attributes1 = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
men_means1 = df_unique_waves_other[df_unique_waves_other['gender'] == 1][attributes1].mean()
women_means1 = df_unique_waves_other[df_unique_waves_other['gender'] == 0][attributes1].mean()

# What opposite sex look for
attributes3 = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
men_means3 = df_unique_waves_other[df_unique_waves_other['gender'] == 1][attributes3].mean()
women_means3 = df_unique_waves_other[df_unique_waves_other['gender'] == 0][attributes3].mean()

# What other fellows look for
attributes2 = ['attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1']
men_means2 = df_unique_waves_1021[df_unique_waves_1021['gender'] == 1][attributes2].mean()
women_means2 = df_unique_waves_1021[df_unique_waves_1021['gender'] == 0][attributes2].mean()

# Calculate base counts
base_women_other = len(df_unique_waves_other[df_unique_waves_other['gender'] == 0])
base_men_other = len(df_unique_waves_other[df_unique_waves_other['gender'] == 1])
base_women_1021 = len(df_unique_waves_1021[df_unique_waves_1021['gender'] == 0])
base_men_1021 = len(df_unique_waves_1021[df_unique_waves_1021['gender'] == 1])

# Create figure with 1 row, 3 columns
fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=['What participants look for', 'What they think fellow men/women look for', 'What they think opposite sex look for'],
    specs=[[{'type': 'polar'}, {'type': 'polar'}, {'type': 'polar'}]]
)

# Add first plot 
fig.add_trace(go.Scatterpolar(
    r=women_means1.values.tolist() + [women_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    mode='lines+markers',  
    name='Women',
    marker_color='red'
), row=1, col=1)

fig.add_trace(go.Scatterpolar(
    r=men_means1.values.tolist() + [men_means1.values[0]],  
    theta=attributes1 + [attributes1[0]],  
    mode='lines+markers',  
    name='Men',
    marker_color='blue'
), row=1, col=1)

# Add second plot 
fig.add_trace(go.Scatterpolar(
    r=women_means3.values.tolist() + [women_means3.values[0]],  
    theta=attributes3 + [attributes3[0]],  
    mode='lines+markers',  
    name='Women',
    marker_color='red',
    showlegend=False  
), row=1, col=2)

fig.add_trace(go.Scatterpolar(
    r=men_means3.values.tolist() + [men_means3.values[0]],  
    theta=attributes3 + [attributes3[0]],  
    mode='lines+markers',  
    name='Men',
    marker_color='blue',
    showlegend=False 
), row=1, col=2)

# Add tird plot 
fig.add_trace(go.Scatterpolar(
    r=women_means2.values.tolist() + [women_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    mode='lines+markers',  
    name='Women',
    marker_color='red',
    showlegend=False  
), row=1, col=3)

fig.add_trace(go.Scatterpolar(
    r=men_means2.values.tolist() + [men_means2.values[0]],  
    theta=attributes2 + [attributes2[0]],  
    mode='lines+markers',  
    name='Men',
    marker_color='blue',
    showlegend=False 
), row=1, col=3)

# Add annotations for base counts at the bottom left
fig.add_annotation(
    text=f"Base Women waves 1-5 and 10-21: {base_women_other}",
    xref="paper", yref="paper",
    x=0.0, y=-0.12,  # Position at the bottom left
    showarrow=False,
    font=dict(size=12),
    align="left"
)

fig.add_annotation(
    text=f"Base Men waves 1-5 and 10-21: {base_men_other}",
    xref="paper", yref="paper",
    x=0.0, y=-0.17,  # Position below the first annotation
    showarrow=False,
    font=dict(size=12),
    align="left"
)

# Add annotations for base counts at the bottom left
fig.add_annotation(
    text=f"Base Women waves 1-5 and 10-21: {base_women_other}",
    xref="paper", yref="paper",
    x=0.5, y=-0.12,  # Position at the bottom left
    showarrow=False,
    font=dict(size=12),
    align="left"
)

fig.add_annotation(
    text=f"Base Men waves 1-5 and 10-21: {base_men_other}",
    xref="paper", yref="paper",
    x=0.5, y=-0.17,  # Position below the first annotation
    showarrow=False,
    font=dict(size=12),
    align="left"
)

# Add annotations for base counts at the bottom right
fig.add_annotation(
    text=f"Base Women waves 10-21: {base_women_1021}",
    xref="paper", yref="paper",
    x=1.0, y=-0.12,  # Position at the bottom right
    showarrow=False,
    font=dict(size=12),
    align="left"
)

fig.add_annotation(
    text=f"Base Men waves 10-21: {base_men_1021}",
    xref="paper", yref="paper",
    x=1.0, y=-0.17,  # Position below the first annotation
    showarrow=False,
    font=dict(size=12),
    align="left"
)


# Update layout
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 40]  
        )
    ),
    polar2=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 40]  
        )
    ),
    polar3=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 40]  
        )
    ),
    showlegend=True,  # This ensures the legend will only be shown once
    title="Comparison of Perceptions<br><sup> 1=awful, 100=great. Total must equal 100</sup>",
    width=1450,
    height=500
)

fig.show()

In [156]:
# Gender mapping
gender_mapping = {0: 'Women', 1: 'Men'}
df_unique['gender_label'] = df_unique['gender'].map(gender_mapping)

# Create subplots
fig = make_subplots(
    rows=1, cols=2, 
    subplot_titles=(
        "Dating someone of same race/ethnic background", 
        "Dating someone of same religious background"
    )
)

# Add first boxplot: Importance of race
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Women']['imprace'],
        name='Women',
        marker_color='red',
        boxmean='sd'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Men']['imprace'],
        name='Men',
        marker_color='blue',
        boxmean='sd'
    ),
    row=1, col=1
)


# Add second boxplot: Importance of religion
fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Women']['imprelig'],
        name='Women',
        marker_color='red',
        boxmean='sd',
        showlegend=False  # Hide duplicate legend
    ),
    row=1, col=2
)

fig.add_trace(
    go.Box(
        y=df_unique[df_unique['gender_label'] == 'Men']['imprelig'],
        name='Men',
        marker_color='blue',
        boxmean='sd',
        showlegend=False
    ),
    row=1, col=2
)

# Update layout with correct Y axis scale
fig.update_layout(
    title_text="Importance of Race and Religion in Dating by Gender<br><sub>Base: 277 men, 274 women.",
    height=500,
    width=1000,
    yaxis=dict(
        title='Importance',
        tickmode='linear',
        dtick=1,
        range=[-0.5, 10.5]   
    ),
    yaxis2=dict(
        title='Importance',
        tickmode='linear',
        dtick=1,
        range=[-0.5, 10.5]  
    ),
)

fig.show()


# Match analysis

In [28]:
# Calculer le taux de match (moyenne) par wave et race
taux_match_par_wave_genre = df.groupby(['wave', 'samerace'])['match'].mean().reset_index()

# Mapper pour lisibilité
taux_match_par_wave_genre['samerace'] = taux_match_par_wave_genre['samerace'].map({0: 'Differente race', 1: 'Same race'})

# Barplot
fig = px.bar(
    taux_match_par_wave_genre,
    x='wave',
    y='match',
    color='samerace',
    barmode='group',
    labels={'wave': 'Wave', 'match': 'Taux de match', 'samerace': 'Race'},
    title='Taux de match (match=1) par wave et par race',
    hover_data={'wave': True, 'samerace': True, 'match': ':.2f'},
    color_discrete_map={'Same race': 'blue', 'Differente race': 'red'}
)

fig.show()


In [35]:
# 1. Créer les deux sous-ensembles
df_samerace = df[df['samerace'] == 1]
df_differentrace = df[df['samerace'] == 0]

# 2. Pour chacun, compter match vs no match
match_counts_samerace = df_samerace['match_alphab'].value_counts().reset_index()
match_counts_samerace.columns = ['Match Type', 'Count']

match_counts_differentrace = df_differentrace['match_alphab'].value_counts().reset_index()
match_counts_differentrace.columns = ['Match Type', 'Count']

# 3. Créer deux pie charts dans une même figure
fig = go.Figure()

# Pie chart pour samerace = 1
fig.add_trace(go.Pie(
    labels=match_counts_samerace['Match Type'],
    values=match_counts_samerace['Count'],
    domain={'x': [0, 0.48]},  # place sur la moitié gauche
    title="Couples of same Race"
))

# Pie chart pour samerace = 0
fig.add_trace(go.Pie(
    labels=match_counts_differentrace['Match Type'],
    values=match_counts_differentrace['Count'],
    domain={'x': [0.52, 1]},  # place sur la moitié droite
    title="Couples of different Race"
))

# 4. Layout global
fig.update_layout(
    title_text="Distribution des Matchs : Couples of same Race vs Different Race",
)

# Affichage
fig.show()